In [21]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
# from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import timedelta
import os
import logging
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [22]:
def load_and_prepare_data(stock_csv, balance_csv, cashflow_csv, income_csv, ticker):
    """
    Load and preprocess stock and financial data, aligning financial metrics by year.
    """
    try:
        # Load stock data
        stock_df = pd.read_csv(stock_csv)
        stock_df['Date'] = pd.to_datetime(stock_df['Date']).dt.date
        stock_df = stock_df[stock_df['Ticker'] == ticker][['Date', 'Close', 'Volume']].sort_values('Date')

        
        
        # Create daily date range and fill missing dates
        date_range = pd.date_range(start=stock_df['Date'].min(), end=stock_df['Date'].max(), freq='D')
        stock_df = stock_df.set_index('Date').reindex(date_range, method='ffill').reset_index()
        stock_df = stock_df.rename(columns={'index': 'Date'})
        stock_df['Ticker'] = ticker
        stock_df['Year'] = pd.to_datetime(stock_df['Date']).dt.year
        
        logging.info(f"Loaded {len(stock_df)} daily stock data points for {ticker}")
        
        # Load financial data
        balance_df = pd.read_csv(balance_csv)
        cashflow_df = pd.read_csv(cashflow_csv)
        income_df = pd.read_csv(income_csv)
        
        # Get available years (excluding index, Ticker)
        year_cols = [col for col in balance_df.columns if col not in ['index', 'Ticker']]
        
        # Select key financial metrics for all years
        financial_metrics = {}
        for metric, source_df in [
            ('Diluted EPS', income_df),
            ('EBITDA', income_df),
            ('Free Cash Flow', cashflow_df),
            ('Net Debt', balance_df)
        ]:
            metric_row = source_df[source_df['index'] == metric]
            if metric_row.empty:
                logging.warning(f"No data for {metric}")
                continue
            # Extract values for all years
            values = metric_row[year_cols].iloc[0].to_dict()
            financial_metrics[metric] = values
        
        if not financial_metrics:
            logging.error("No valid financial metrics available.")
            return None, None
        
        # Create financial DataFrame with year-based values
        financial_df = pd.DataFrame([
            {'Year': int(year), **{metric: values.get(year, np.nan) for metric, values in financial_metrics.items()}}
            for year in year_cols
        ])
        logging.info(f"Financial data years: {financial_df['Year'].unique()}")
        logging.info(f"Financial data sample:\n{financial_df.head().to_string()}")
        
        # Merge with stock data by year
        merged_df = stock_df.merge(financial_df, on='Year', how='left')
        merged_df = merged_df.set_index('Date').asfreq('D')  # Set daily frequency
        
        # Forward-fill financial metrics
        financial_cols = ['Diluted EPS', 'Free Cash Flow', 'Net Debt', 'EBITDA']
        available_cols = [col for col in financial_cols if col in merged_df.columns]
        if not available_cols:
            logging.error("No financial columns available after merge.")
            return None, None
        
        merged_df[available_cols] = merged_df[available_cols].ffill().fillna(merged_df[available_cols].mean())
        
        # Validate for NaN and inf
        if merged_df[available_cols].isna().any().any():
            logging.error("NaN values remain in financial columns after filling.")
            return None, None
        if merged_df[available_cols].apply(lambda x: np.isinf(x)).any().any():
            logging.error("Inf values in financial columns.")
            return None, None
        
        # Check variance
        for col in available_cols + ['Close', 'Volume']:
            if merged_df[col].var() < 1e-10:
                logging.warning(f"Column '{col}' has near-zero variance: {merged_df[col].var()}. Adding noise.")
                merged_df[col] += np.random.normal(0, 0.01, len(merged_df))
        
        # Scale financial features and Close
        # scaler = StandardScaler()
        # scale_cols = ['Close', 'Volume'] + available_cols
        # merged_df[scale_cols] = scaler.fit_transform(merged_df[scale_cols])
        
        # Final validation
        # if merged_df[scale_cols].isna().any().any() or merged_df[scale_cols].apply(lambda x: np.isinf(x)).any().any():
        #     logging.error("NaN or inf values after scaling.")
        #     return None, None
        
        logging.info(f"Prepared data with {len(merged_df)} rows, columns: {list(merged_df.columns)}")
        # logging.info(f"Variances after scaling: {merged_df[scale_cols].var().to_dict()}")
        return merged_df #, scaler
    
    except FileNotFoundError as e:
        logging.error(f"File not found: {e}")
        return None, None
    except Exception as e:
        logging.error(f"Error preparing data: {e}")
        return None, None

In [23]:
def calculate_metrics(actual, predicted):
    """
    Calculate RMSE, MAE, and MAPE metrics.
    """
    actual = np.array(actual)
    predicted = np.array(predicted)
    rmse = np.sqrt(np.mean((actual - predicted) ** 2))
    mae = np.mean(np.abs(actual - predicted))
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100 if np.all(actual != 0) else np.nan
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

In [24]:
def arima_forecast(data, forecast_horizon=7):
    """
    Fit ARIMA model using auto_arima and forecast future prices.
    """
    try:
        # Tune ARIMA
        model = auto_arima(data, seasonal=False, max_p=5, max_q=5, max_d=2, 
                          stepwise=True, trace=True, error_action='ignore')
        best_order = model.order
        logging.info(f"Best ARIMA order: {best_order}")
        
        # Fit ARIMA
        arima_model = ARIMA(data, order=best_order)
        model_fit = arima_model.fit()
        
        # Forecast
        forecast = model_fit.forecast(steps=forecast_horizon)
        
        # Evaluate
        if len(data) >= forecast_horizon:
            test_data = data[-forecast_horizon:]
            forecast_test = model_fit.forecast(steps=forecast_horizon)[-forecast_horizon:]
            metrics = calculate_metrics(test_data, forecast_test)
        else:
            metrics = {'RMSE': np.nan, 'MAE': np.nan, 'MAPE': np.nan}
        
        logging.info(f"ARIMA Metrics: {metrics}")
        return forecast, metrics, best_order
    except Exception as e:
        logging.error(f"Error in ARIMA forecasting: {e}")
        return None, None, None

In [25]:
def prophet_forecast(data, forecast_horizon=7, changepoint_prior_scale=0.05):
    """
    Fit Prophet model with scaled financial regressors.
    """
    try:
        # Log input DataFrame columns
        logging.info(f"Prophet input DataFrame columns: {list(data.columns)}")
        
        # Check for required columns  
        prophet_df = data.reset_index()[['Date', 'Close', 'Diluted EPS', 'Free Cash Flow', 'Net Debt', 'EBITDA']]
        prophet_df = prophet_df.rename(columns={'Date': 'ds', 'Close': 'y'})
        
        # Log input data for debugging
        logging.info(f"Prophet input data summary:\n{prophet_df.describe()}")
        
        # Fit Prophet model
        model = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True,
                       changepoint_prior_scale=changepoint_prior_scale, mcmc_samples=0)
        for regressor in ['Diluted EPS', 'Free Cash Flow', 'Net Debt', 'EBITDA']:
            model.add_regressor(regressor)
        model.fit(prophet_df)
        
        # Create future dataframe
        future = model.make_future_dataframe(periods=forecast_horizon)
        for regressor in ['Diluted EPS', 'Free Cash Flow', 'Net Debt', 'EBITDA']:
            future[regressor] = prophet_df[regressor].iloc[-1]
        forecast_df = model.predict(future)
        
        # Extract forecast
        forecast = forecast_df[['ds', 'yhat']].tail(forecast_horizon).set_index('ds')['yhat']
        
        # Evaluate
        test_data = prophet_df['y'][-forecast_horizon:]
        forecast_test = model.predict(prophet_df[-forecast_horizon:])['yhat']
        metrics = calculate_metrics(test_data, forecast_test)

        
        logging.info(f"Prophet Metrics (changepoint_prior_scale={changepoint_prior_scale}): {metrics}")
        return forecast, metrics, forecast_df
    except Exception as e:
        logging.error(f"Error in Prophet forecasting: {e}")
        return None, None, None

In [26]:
def tune_prophet(data, forecast_horizon=7):
    """
    Tune Prophet model by testing changepoint_prior_scale values.
    """
    scales = [0.05, 0.1, 0.5]
    best_metrics = {'RMSE': float('inf')}
    best_forecast = None
    best_forecast_df = None
    best_scale = 0.05
    
    for scale in scales:
        forecast, metrics, forecast_df = prophet_forecast(data, forecast_horizon, changepoint_prior_scale=scale)
        if forecast is not None and not np.any(np.isnan(forecast)) and metrics['RMSE'] < best_metrics['RMSE']:
            best_metrics = metrics
            best_forecast = forecast
            best_forecast_df = forecast_df
            best_scale = scale
    
    if best_forecast is None:
        logging.warning("All Prophet models failed. Using default scale=0.05.")
        forecast, metrics, forecast_df = prophet_forecast(data, forecast_horizon, changepoint_prior_scale=0.05)
        best_metrics = metrics if metrics else {'RMSE': np.nan, 'MAE': np.nan, 'MAPE': np.nan}
        best_forecast = forecast
        best_forecast_df = forecast_df
        best_scale = 0.05
    
    logging.info(f"Best Prophet changepoint_prior_scale: {best_scale}")
    logging.info(f"Best Prophet Metrics: {best_metrics}")
    return best_forecast, best_metrics, best_forecast_df, best_scale

In [27]:
def backtest_strategy(data, predictions, model_name, threshold=0.01):
    """
    Backtest a trading strategy with a threshold.
    """
    try:
        # Validate input lengths
        logging.info(f"Backtest input lengths - Data: {len(data)}, Predictions: {len(predictions)}")
        if len(predictions) != len(data):
            logging.warning(f"Predictions length ({len(predictions)}) does not match data length ({len(data)}). Truncating to match.")
            min_length = min(len(data), len(predictions))
            data = data.iloc[:min_length]
            predictions = predictions[:min_length]
        
        pred_df = pd.DataFrame({'Date': data.index, 'Close': data['Close'], 'Prediction': predictions})
        pred_df = pred_df.dropna()
        logging.info(f"pred_df shape after dropna: {pred_df.shape}, index type: {type(pred_df.index)}")
        
        # Generate signals
        pred_df['Signal'] = 0
        pred_df.iloc[1:, pred_df.columns.get_loc('Signal')] = np.where(
            pred_df['Prediction'].shift(-1)[1:] > pred_df['Prediction'][1:] * (1 + threshold), 1,
            np.where(pred_df['Prediction'].shift(-1)[1:] < pred_df['Prediction'][1:] * (1 - threshold), -1, 0)
        )
        
        # Calculate returns
        pred_df['Return'] = pred_df['Close'].pct_change()
        pred_df['Strategy_Return'] = pred_df['Signal'].shift(1) * pred_df['Return']
        
        # Cumulative return
        cumulative_return = (1 + pred_df['Strategy_Return'].dropna()).cumprod().iloc[-1] - 1
        num_trades = pred_df['Signal'].abs().sum()
        
        results = {
            'Cumulative Return (%)': cumulative_return * 100,
            'Number of Trades': num_trades
        }
        
        logging.info(f"Backtest Results for {model_name}: {results}")
        return results, pred_df
    except Exception as e:
        logging.error(f"Error in backtesting: {e}")
        return None, None

In [28]:
def walk_forward_validation(data, forecast_horizon=7, n_folds=5):
    """
    Perform walk-forward validation for ARIMA and Prophet.
    """
    arima_metrics_list = []
    prophet_metrics_list = []
    
    for i in range(n_folds):
        train_end = len(data) - (n_folds - i) * forecast_horizon
        if train_end <= forecast_horizon:
            continue
        
        train_data = data.iloc[:train_end]
        test_data = data.iloc[train_end:train_end + forecast_horizon]['Close']
        
        if len(test_data) != forecast_horizon:
            continue
        
        # ARIMA
        arima_model = auto_arima(train_data['Close'], seasonal=False, max_p=5, max_q=5, max_d=2,
                                stepwise=True, error_action='ignore')
        arima_fit = ARIMA(train_data['Close'], order=arima_model.order).fit()
        arima_pred = arima_fit.forecast(steps=forecast_horizon)
        arima_metrics = calculate_metrics(test_data, arima_pred)
        arima_metrics_list.append(arima_metrics)
        
        # Prophet
        prophet_df = train_data.reset_index()[['Date', 'Close', 'Diluted EPS', 'Free Cash Flow', 'Net Debt', 'EBITDA']]
        prophet_df = prophet_df.rename(columns={'Date': 'ds', 'Close': 'y'})

        
        prophet_model = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True,
                               changepoint_prior_scale=0.05, mcmc_samples=0)
        for regressor in ['Diluted EPS', 'Free Cash Flow', 'Net Debt', 'EBITDA']:
            prophet_model.add_regressor(regressor)
        prophet_model.fit(prophet_df)
        future = prophet_model.make_future_dataframe(periods=forecast_horizon)
        for regressor in ['Diluted EPS', 'Free Cash Flow', 'Net Debt', 'EBITDA']:
            future[regressor] = prophet_df[regressor].iloc[-1]
        prophet_pred_df = prophet_model.predict(future)
        prophet_pred = prophet_pred_df['yhat'].tail(forecast_horizon)
        prophet_metrics = calculate_metrics(test_data, prophet_pred)
        prophet_metrics_list.append(prophet_metrics)
    
    # Average metrics
    avg_metrics = {
        'ARIMA': {
            'RMSE': np.mean([m['RMSE'] for m in arima_metrics_list]) if arima_metrics_list else np.nan,
            'MAE': np.mean([m['MAE'] for m in arima_metrics_list]) if arima_metrics_list else np.nan,
            'MAPE': np.mean([m['MAPE'] for m in arima_metrics_list if not np.isnan(m['MAPE'])]) if any(not np.isnan(m['MAPE']) for m in arima_metrics_list) else np.nan
        },
        'Prophet': {
            'RMSE': np.mean([m['RMSE'] for m in prophet_metrics_list]) if prophet_metrics_list else np.nan,
            'MAE': np.mean([m['MAE'] for m in prophet_metrics_list]) if prophet_metrics_list else np.nan,
            'MAPE': np.mean([m['MAPE'] for m in prophet_metrics_list if not np.isnan(m['MAPE'])]) if any(not np.isnan(m['MAPE']) for m in prophet_metrics_list) else np.nan
        }
    }
    
    logging.info(f"Walk-Forward Validation Results: {avg_metrics}")
    return avg_metrics

In [139]:
def plot_predictions(data, arima_forecast, prophet_forecast, prophet_forecast_df, backtest_arima_df, backtest_prophet_df, forecast_horizon=7):
    """
    Plot actual vs predicted prices and backtest signals in a 1x2 subplot: ARIMA on left, Prophet on right.
    Optimized for faster rendering and image export, with range sliders on both subplots.
    """
    last_date = data.index[-1]
    future_dates = [last_date + timedelta(days=i+1) for i in range(forecast_horizon)]
    
    # Create 1x2 subplot with shared y-axes
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            'ARIMA: AAPL Stock Price Prediction and Backtest Signals',
            'Prophet: AAPL Stock Price Prediction and Backtest Signals'
        ),
        shared_yaxes=True
    )
    
    # --- ARIMA Subplot (Left, Column 1) ---
    # Historical Close
    fig.add_trace(
        go.Scatter(x=data.index, y=data['Close'], name='Historical Close', line=dict(color='blue')),
        row=1, col=1
    )
    
    # ARIMA Forecast
    if arima_forecast is not None:
        fig.add_trace(
            go.Scatter(x=future_dates, y=arima_forecast, name='ARIMA Forecast',
                       line=dict(color='red', dash='dash')),
            row=1, col=1
        )
    
    # ARIMA Buy/Sell Signals (single trace)
    if backtest_arima_df is not None:
        signals = backtest_arima_df[backtest_arima_df['Signal'].isin([1, -1])]
        marker_colors = ['green' if s == 1 else 'red' for s in signals['Signal']]
        marker_symbols = ['triangle-up' if s == 1 else 'triangle-down' for s in signals['Signal']]
        fig.add_trace(
            go.Scatter(
                x=signals['Date'],
                y=signals['Close'],
                name='ARIMA Signals',
                mode='markers',
                marker=dict(
                    color=marker_colors,
                    symbol=marker_symbols,
                    size=10,
                    opacity=0.7
                ),
                text=['Buy' if s == 1 else 'Sell' for s in signals['Signal']],
                hovertemplate='%{text}<br>Date: %{x}<br>Close: %{y:.2f}'
            ),
            row=1, col=1
        )
    
    # --- Prophet Subplot (Right, Column 2) ---
    # Historical Close
    fig.add_trace(
        go.Scatter(x=data.index, y=data['Close'], name='Historical Close', line=dict(color='blue'), showlegend=False),
        row=1, col=2
    )
    
    # Prophet Forecast
    if prophet_forecast is not None:
        fig.add_trace(
            go.Scatter(x=future_dates, y=prophet_forecast, name='Prophet Forecast',
                       line=dict(color='green', dash='dash'), ),
            row=1, col=2
        )
        # Confidence Interval
        if prophet_forecast_df is not None and 'yhat_upper' in prophet_forecast_df.columns:
            fig.add_trace(
                go.Scatter(
                    x=future_dates + future_dates[::-1],
                    y=list(prophet_forecast_df['yhat_upper'].tail(forecast_horizon)) +
                      list(prophet_forecast_df['yhat_lower'].tail(forecast_horizon))[::-1],
                    fill='toself',
                    fillcolor='rgba(0, 128, 0, 0.1)',
                    line=dict(color='rgba(255,255,255,0)'),
                    name='Prophet Confidence Interval',
                    hoverinfo='skip'
                ),
                row=1, col=2
            )
    
    # Prophet Buy/Sell Signals (single trace)
    if backtest_prophet_df is not None:
        signals = backtest_prophet_df[backtest_prophet_df['Signal'].isin([1, -1])]
        marker_colors = ['lime' if s == 1 else 'darkred' for s in signals['Signal']]
        marker_symbols = ['triangle-up' if s == 1 else 'triangle-down' for s in signals['Signal']]
        fig.add_trace(
            go.Scatter(
                x=signals['Date'],
                y=signals['Close'],
                name='Prophet Signals',
                mode='markers',
                marker=dict(
                    color=marker_colors,
                    symbol=marker_symbols,
                    size=10,
                    opacity=0.7
                ),
                text=['Buy' if s == 1 else 'Sell' for s in signals['Signal']],
                hovertemplate='%{text}<br>Date: %{x}<br>Close: %{y:.2f}'
            ),
            row=1, col=2
        )
    
    # Update layout
    fig.update_layout(
        title=dict(text='AAPL Stock Price Prediction and Backtest Signals (7-Day Forecast)',
                   x=0.5, xanchor='center', y=0.98),
        height=1000, width=1400,
        showlegend=True,
        legend=dict(orientation='h', yanchor='bottom', y=-0.35, xanchor='center', x=0.5),
        hovermode='x unified',
        plot_bgcolor='white',
        margin=dict(l=50, r=50, t=100, b=100)
    )
    
    # Update axes
    fig.update_xaxes(title_text='Date', gridcolor='lightgrey', row=1, col=1)
    fig.update_yaxes(title_text='Close Price (USD)', gridcolor='lightgrey', row=1, col=1)
    fig.update_xaxes(title_text='Date', gridcolor='lightgrey', row=1, col=2)
    
    # Add range slider (on both subplots)
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeslider_thickness=0.15,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1mo', step='month', stepmode='backward'),
                dict(count=6, label='6mo', step='month', stepmode='backward'),
                dict(count=1, label='1y', step='year', stepmode='backward'),
                dict(step='all', label='All'),
            ]),
            x=0.055,
            y=0.95,
        ),
        row=1, col=1
    )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeslider_thickness=0.15,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1mo', step='month', stepmode='backward'),
                dict(count=6, label='6mo', step='month', stepmode='backward'),
                dict(count=1, label='1y', step='year', stepmode='backward'),
                dict(step='all', label='All'),
            ]),
            x=0.595,
            y=0.95,
        ),
        row=1, col=2
    )
    
    fig.show()

**Main**

In [30]:
# Parameters
ticker = pd.read_csv('../data/cleaned_stock_data.csv')['Ticker'].unique()[0] 
forecast_horizon = 7
stock_csv = '../data/cleaned_stock_data.csv'
balance_csv = f'../data/balance_sheet_{ticker}.csv'
cashflow_csv = f'../data/cash_flow_{ticker}.csv'
income_csv = f'../data/income_statement_{ticker}.csv'

# Configure cmdstanpy
cmdstan_dir = os.path.expanduser('C:/Users/nguye/.cmdstan')
os.makedirs(cmdstan_dir, exist_ok=True)
os.environ['CMDSTAN'] = cmdstan_dir
logging.info(f"Set CMDSTAN directory to {cmdstan_dir}")

# Load and prepare data
data = load_and_prepare_data(stock_csv, balance_csv, cashflow_csv, income_csv, ticker)

2025-05-06 08:00:45,989 - INFO - Set CMDSTAN directory to C:/Users/nguye/.cmdstan
2025-05-06 08:00:46,006 - INFO - Loaded 725 daily stock data points for AAPL
2025-05-06 08:00:46,016 - INFO - Financial data years: [2024 2023 2022 2021 2020]
2025-05-06 08:00:46,018 - INFO - Financial data sample:
   Year  Diluted EPS        EBITDA  Free Cash Flow      Net Debt
0  2024         6.08  1.346610e+11    1.088070e+11  7.668600e+10
1  2023         6.13  1.258200e+11    9.958400e+10  8.112300e+10
2  2022         6.11  1.305410e+11    1.114430e+11  9.642300e+10
3  2021         5.61  1.231360e+11    9.295300e+10  8.977900e+10
4  2020          NaN           NaN             NaN           NaN
2025-05-06 08:00:46,026 - INFO - Prepared data with 725 rows, columns: ['Close', 'Volume', 'Ticker', 'Year', 'Diluted EPS', 'EBITDA', 'Free Cash Flow', 'Net Debt']


**Models**

In [31]:
# ARIMA forecast
arima_forecast, arima_metrics, arima_order = arima_forecast(data['Close'], forecast_horizon)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=3636.581, Time=0.34 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3652.778, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3644.135, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3643.507, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3650.950, Time=0.01 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=3641.079, Time=0.17 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3638.932, Time=0.15 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=3637.470, Time=0.34 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=3637.055, Time=0.34 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3645.503, Time=0.08 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=3635.178, Time=0.17 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=3634.581, Time=0.09 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3645.491, Time=0.09 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=3634.731, Time=0.09 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept

2025-05-06 08:00:48,786 - INFO - Best ARIMA order: (0, 1, 3)
2025-05-06 08:00:48,852 - INFO - ARIMA Metrics: {'RMSE': 5.6162121721201, 'MAE': 5.101869037138881, 'MAPE': 2.4152470114619238}


 ARIMA(1,1,4)(0,0,0)[0]             : AIC=3633.902, Time=0.16 sec

Best model:  ARIMA(0,1,3)(0,0,0)[0]          
Total fit time: 2.741 seconds


In [32]:
# Prophet forecast with tuning
prophet_forecast, prophet_metrics, prophet_forecast_df, prophet_scale = tune_prophet(data, forecast_horizon)

2025-05-06 08:00:48,872 - INFO - Prophet input DataFrame columns: ['Close', 'Volume', 'Ticker', 'Year', 'Diluted EPS', 'EBITDA', 'Free Cash Flow', 'Net Debt']
2025-05-06 08:00:48,890 - INFO - Prophet input data summary:
                        ds           y  Diluted EPS  Free Cash Flow  \
count                  725  725.000000   725.000000    7.250000e+02   
mean   2024-05-05 00:00:00  202.310303     6.096345    1.057920e+11   
min    2023-05-09 00:00:00  165.000000     6.080000    9.958400e+10   
25%    2023-11-06 00:00:00  180.950000     6.080000    9.958400e+10   
50%    2024-05-05 00:00:00  193.580000     6.080000    1.088070e+11   
75%    2024-11-02 00:00:00  225.890000     6.130000    1.088070e+11   
max    2025-05-02 00:00:00  259.020000     6.130000    1.088070e+11   
std                    NaN   24.825704     0.023470    4.329303e+09   

           Net Debt        EBITDA  
count  7.250000e+02  7.250000e+02  
mean   7.813644e+10  1.317709e+11  
min    7.668600e+10  1.258200e+1

**Walk-forward validation**

In [33]:
avg_metrics = walk_forward_validation(data, forecast_horizon)

2025-05-06 08:00:51,500 - DEBUG - cmd: where.exe tbb.dll
cwd: None
2025-05-06 08:00:51,550 - DEBUG - TBB already found in load path
2025-05-06 08:00:51,567 - DEBUG - input tempfile: C:\Users\nguye\AppData\Local\Temp\tmp8cjqu3hw\krqsehpy.json
2025-05-06 08:00:51,619 - DEBUG - input tempfile: C:\Users\nguye\AppData\Local\Temp\tmp8cjqu3hw\h0x2fi8n.json
2025-05-06 08:00:51,622 - DEBUG - idx 0
2025-05-06 08:00:51,622 - DEBUG - running CmdStan, num_threads: None
2025-05-06 08:00:51,622 - DEBUG - CmdStan args: ['C:\\Users\\nguye\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=7606', 'data', 'file=C:\\Users\\nguye\\AppData\\Local\\Temp\\tmp8cjqu3hw\\krqsehpy.json', 'init=C:\\Users\\nguye\\AppData\\Local\\Temp\\tmp8cjqu3hw\\h0x2fi8n.json', 'output', 'file=C:\\Users\\nguye\\AppData\\Local\\Temp\\tmp8cjqu3hw\\prophet_modelw_a5swfh\\prophet_model-20250506080051.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:00

**Backtesting**

In [34]:
if len(data) >= forecast_horizon:
    # ARIMA historical predictions
    arima_model = auto_arima(data['Close'][:-forecast_horizon], seasonal=False, max_p=5, max_q=5, max_d=2)
    arima_fit = ARIMA(data['Close'][:-forecast_horizon], order=arima_model.order).fit()
    arima_hist_pred = arima_fit.predict(start=0, end=len(data)-forecast_horizon-1)
    arima_backtest_results, arima_backtest_df = backtest_strategy(data.iloc[:-forecast_horizon], arima_hist_pred, 'ARIMA')
    
    # Prophet historical predictions
    train_data = data.iloc[:-forecast_horizon]
    prophet_df = train_data.reset_index()[['Date', 'Close', 'Diluted EPS', 'Free Cash Flow', 'Net Debt', 'EBITDA']]
    prophet_df = prophet_df.rename(columns={'Date': 'ds', 'Close': 'y'})
    
    # Remove duplicate dates
    duplicate_ds = prophet_df['ds'].duplicated().sum()
    if duplicate_ds > 0:
        logging.warning(f"Found {duplicate_ds} duplicate dates in backtest prophet_df. Removing duplicates.")
        prophet_df = prophet_df.drop_duplicates(subset='ds')
    logging.info(f"Prophet backtest input length: {len(prophet_df)}, Train data length: {len(train_data)}")
    
    prophet_model = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True,
                            changepoint_prior_scale=0.5, mcmc_samples=0)
    for regressor in ['Diluted EPS', 'Free Cash Flow', 'Net Debt', 'EBITDA']:
        prophet_model.add_regressor(regressor)
    prophet_model.fit(prophet_df)
    
    # Predict and reindex to match train_data
    prophet_pred_df = prophet_model.predict(prophet_df)
    prophet_pred_df = prophet_pred_df.set_index('ds').reindex(train_data.index, method='ffill')
    prophet_hist_pred = prophet_pred_df['yhat']
    logging.info(f"Prophet backtest predictions length: {len(prophet_hist_pred)}")
    
    # Log prediction variability
    pred_diff = prophet_hist_pred.shift(-1) - prophet_hist_pred
    logging.info(f"Prophet predictions max diff: {pred_diff.max()}, min diff: {pred_diff.min()}, mean diff: {pred_diff.mean()}")
    
    prophet_backtest_results, prophet_backtest_df = backtest_strategy(train_data, prophet_hist_pred, 'Prophet', threshold=0.005)
else:
    arima_backtest_results, arima_backtest_df = None, None
    prophet_backtest_results, prophet_backtest_df = None, None
    logging.warning("Insufficient data for backtesting.")

2025-05-06 08:01:10,599 - INFO - Backtest input lengths - Data: 718, Predictions: 718
2025-05-06 08:01:10,602 - INFO - pred_df shape after dropna: (718, 3), index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
2025-05-06 08:01:10,619 - INFO - Backtest Results for ARIMA: {'Cumulative Return (%)': 47.34550965812281, 'Number of Trades': 225}
2025-05-06 08:01:10,624 - INFO - Prophet backtest input length: 718, Train data length: 718
2025-05-06 08:01:10,627 - DEBUG - cmd: where.exe tbb.dll
cwd: None
2025-05-06 08:01:10,709 - DEBUG - TBB already found in load path
2025-05-06 08:01:10,739 - DEBUG - input tempfile: C:\Users\nguye\AppData\Local\Temp\tmp8cjqu3hw\f366xh7y.json
2025-05-06 08:01:10,851 - DEBUG - input tempfile: C:\Users\nguye\AppData\Local\Temp\tmp8cjqu3hw\widmf58c.json
2025-05-06 08:01:10,855 - DEBUG - idx 0
2025-05-06 08:01:10,856 - DEBUG - running CmdStan, num_threads: None
2025-05-06 08:01:10,857 - DEBUG - CmdStan args: ['C:\\Users\\nguye\\AppData\\Local\\Programs\

**Plot predictions**

In [140]:
# Plot predictions and backtest signals
plot_predictions(data, arima_forecast, prophet_forecast, prophet_forecast_df, 
                arima_backtest_df, prophet_backtest_df, forecast_horizon)

In [36]:
# Save predictions
if arima_forecast is not None and prophet_forecast is not None:
    future_dates = [data.index[-1] + timedelta(days=i+1) for i in range(forecast_horizon)]
    pred_df = pd.DataFrame({
        'Date': future_dates,
        'ARIMA_Prediction': arima_forecast,
        'Prophet_Prediction': prophet_forecast
    })
    pred_df.to_csv('stock_price_predictions.csv', index=False)
    logging.info("Saved predictions to stock_price_predictions.csv")

# Save metrics
metrics_df = pd.DataFrame({
    'Model': ['ARIMA', 'Prophet'],
    'RMSE': [arima_metrics.get('RMSE', np.nan), prophet_metrics.get('RMSE', np.nan)],
    'MAE': [arima_metrics.get('MAE', np.nan), prophet_metrics.get('MAE', np.nan)],
    'MAPE': [arima_metrics.get('MAPE', np.nan), prophet_metrics.get('MAPE', np.nan)],
    'Best Parameters': [f"Order: {arima_order}", f"changepoint_prior_scale: {prophet_scale or 0.05}"]
})
metrics_df.to_csv('model_metrics.csv', index=False)
logging.info("Saved metrics to model_metrics.csv")

# Save backtest results
if arima_backtest_results and prophet_backtest_results:
    backtest_df = pd.DataFrame({
        'Model': ['ARIMA', 'Prophet'],
        'Cumulative Return (%)': [arima_backtest_results['Cumulative Return (%)'], 
                                    prophet_backtest_results['Cumulative Return (%)']],
        'Number of Trades': [arima_backtest_results['Number of Trades'], 
                            prophet_backtest_results['Number of Trades']]
    })
    backtest_df.to_csv('backtest_results.csv', index=False)
    logging.info("Saved backtest results to backtest_results.csv")

2025-05-06 08:01:11,948 - INFO - Saved predictions to stock_price_predictions.csv
2025-05-06 08:01:11,958 - INFO - Saved metrics to model_metrics.csv
2025-05-06 08:01:11,962 - INFO - Saved backtest results to backtest_results.csv
